In [1]:
import os
import csv
import json

In [2]:
def focus_unclear_prompt(title):
    return f"""Tóm tắt nội dung chính của cuốn sách '{title}'"""

In [3]:
books_data = []

# Đọc file JSON 400 cuốn sách
with open("data/book_reviews_prompts.csv", "r", encoding="utf-8") as csvfile:
    reader = csv.DictReader(
        csvfile
    )  # Sử dụng DictReader để đọc từng dòng dưới dạng dict
    for row in reader:
        # Giả sử cột chứa tên sách là "title" và "description"
        book_no = row.get(
            "No.", "Unknown No."
        )  # Default to "Unknown No." if the column is missing
        book_title = row.get(
            "book_title", "Unknown Title"
        )  # Default to "Unknown Title" if the column is missing
        book_description = row.get(
            "description", "No Description Available"
        )  # Default to "No Description Available"
        books_data.append(
            {"no.": book_no, "title": book_title, "description": book_description}
        )
books_data

[{'no.': '1',
  'title': 'African Cooking 101: African Recipes Cookbook for Beginners',
  'description': 'African Recipes Cookbook for Beginners": Simple, delicious and healthy recipes from all over Africa\nToday only, get this Amazon bestseller for just $0.99. Regularly priced at $4.99. Read on your PC, Mac, smart phone, tablet or Kindle device.\nIf the topic about African cuisine mystifies you, you would not be alone. Africa is arguably the world\'s most diverse region. It is an entire continent with 55 recognised independent states. When it comes to the food, you would be right to assume that there is more to it than meets the eye. is there such thing as African cuisine?\nAfrica has got a mixture of indigenous foods, with European, Asian and Middle Eastern influences depending on what region you venture into.\nThis can at times make it very difficult to distinguish between indigenous foods and those that have heavy external influences.\nAt the same time, by recognising the culinary 

In [5]:
INPUT_JSON_FILE = "data/book_review_agent_prompts.json"

user_contents = []

if books_data:
    for book in books_data:
        title = book["title"]

        prompt_template = focus_unclear_prompt(title)

        user_contents.append(
            {
                "eval_id": f"eval_case_{int(book['no.']):03d}",
                "user_content": {"parts": [{"text": prompt_template}], "role": "user"},
            }
        )

    print(f"Đã tạo thành công {len(user_contents)} yêu cầu cho batch job.")
else:
    print("Không có sách nào để tạo yêu cầu.")

output_filepath = "data/focus_unclear_contents.json"

os.makedirs(
    os.path.dirname(INPUT_JSON_FILE), exist_ok=True
)  # Tạo thư mục nếu chưa tồn tại

with open(output_filepath, "w", encoding="utf-8") as output_file:
    json.dump(user_contents, output_file, indent=2, ensure_ascii=False)

print(f"Dữ liệu đã được lưu vào {output_filepath}")

Đã tạo thành công 100 yêu cầu cho batch job.
Dữ liệu đã được lưu vào data/focus_unclear_contents.json


In [10]:
# Đường dẫn tệp JSON và CSV
json_filepath = "../book_review_agent/focus_unclear.evalset.json"
focus_unclear_prompts = "data/focus_unclear_contents.json"
output_filepath = "data/focus_unclear.evalset.json"

# Đọc tệp JSON
with open(json_filepath, "r") as json_file:
    data = json.load(json_file)

# Lấy sườn từ eval_case đầu tiên
template_case = data["eval_cases"][0]

# Đọc tệp CSV
with open(focus_unclear_prompts, "r") as json_file:
    focus_unclear_data = json.load(json_file)
    
# Tạo các eval_cases mới từ sườn và dữ liệu CSV
new_eval_cases = []
for i, row in enumerate(focus_unclear_data):
    new_case = json.loads(json.dumps(template_case))  # Tạo bản sao của sườn
    new_case["eval_id"] = row["eval_id"]
    new_case["conversation"][0][
        "invocation_id"
    ] = f"invocation-{i+1}"  # Gán invocation_id mới
    new_case["conversation"][0]["user_content"] = row["user_content"]
    new_eval_cases.append(new_case)

# Cập nhật eval_cases trong dữ liệu gốc
data["eval_cases"] = new_eval_cases

# Ghi tệp JSON mới
with open(output_filepath, "w", encoding="utf-8") as json_file:
    json.dump(data, json_file, indent=2, ensure_ascii=False)

print(f"Parsed eval_cases saved to {output_filepath}")

Parsed eval_cases saved to data/focus_unclear.evalset.json


In [ ]:
import glob

focus_unclear_prompts = "data/focus_unclear.evalset.json"

# Đọc tệp JSON
with open(focus_unclear_prompts, "r") as json_file:
    updated_focus_unclear_final_response = json.load(json_file)
    
# Đường dẫn thư mục chứa các file kết quả (giả sử là JSON)
results_folder = "../book_review_agent/.adk/parsed"
result_files = glob.glob(os.path.join(results_folder, "*.json"))

# Tạo một dict để tra cứu nhanh eval_id -> eval_case
eval_cases_dict = {case["eval_id"]: case for case in updated_focus_unclear_final_response["eval_cases"]}

for file_path in result_files:
    with open(file_path, "r", encoding="utf-8") as f:
        result = json.load(f)
        eval_id = result["eval_case_results"][0]["eval_id"]
        final_response =  result["eval_case_results"][0]["eval_metric_result_per_invocation"][0]["actual_invocation"]["final_response"]
        parts = final_response.get("parts", [])
        if eval_id in eval_cases_dict and parts:
            text = parts[0].get("text", "")
            eval_cases_dict[eval_id]["conversation"][0]["final_response"]["parts"][0]["text"] = text

# Cập nhật eval_cases trong dữ liệu gốc
updated_focus_unclear_final_response["eval_casese"] = eval_cases_dict
updated_output_filepath="data/focus_unclear_updated.eval.json"

# Ghi tệp JSON mới
with open(updated_output_filepath, "w", encoding="utf-8") as json_file:
    json.dump(updated_focus_unclear_final_response, json_file, indent=2, ensure_ascii=False)

print(f"Parsed eval_cases saved to {updated_output_filepath}")

Parsed eval_cases saved to data/focus_unclear_updated.eval.json


### Run evaluation then fetch detail
When complete all the steps above you can run the evaluation then navigate to [Fetch session trace](fetch_session_trace.ipynb) to fetch and analyze data.
